# Word2vec & Word Embeddings

Word2vec se basa en la idea de que el significado de una palabra está definido por su contexto. El contexto se representa como palabras circundantes.

## One hot encoding 

<img src="images/one_hot_encoding.png" alt="drawing" width="500"/>

## CBOW

Un modelo que predice una palabra actual en función de sus palabras de contexto.

## <img src="images/cbow.png" alt="drawing" width="700"/>

## Skip-Gram

Un modelo que predice palabras de contexto en función de la palabra actual.

<img src="images/skipgram.png" alt="drawing" width="700"/>

## Redes neuronales

### Skipgram

<img src="images/word_embedding_architecture.png" width=70% style="float: center;"></img>


### CBoW / Skip-gram

<img src="images/Continuous-Bag-of-words-CBOW-CB-and-Skip-gram-SG-training-model-illustrations.png" width=50% style="float: center;"></img>

## CBoW

In [1]:
import torch
import torch.nn as nn

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMDEDDING_DIM = 100

raw_text = """Elon Musk es un empresario inversor y magnate estadounidense de origen sudafricano. 
Es el fundador, consejero delegado e ingeniero jefe de SpaceX; inversor ángel, CEO y arquitecto de productos de Tesla, Inc; 
fundador de The Boring Company; y cofundador de Neuralink y OpenAI. 
Con un patrimonio neto estimado en unos 252 mil millones de dólares en abril de 2022, Musk es la persona más rica del mundo según 
el índice de multimillonarios de Bloomberg y la lista de multimillonarios en tiempo real de Forbes.""".split()

vocab = set(raw_text)
vocab_size = len(vocab)

In [2]:
word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

In [4]:
ix_to_word

{0: 'ingeniero',
 1: 'Tesla,',
 2: 'SpaceX;',
 3: 'neto',
 4: 'ángel,',
 5: 'según',
 6: 'sudafricano.',
 7: 'un',
 8: 'jefe',
 9: 'Elon',
 10: 'Inc;',
 11: 'unos',
 12: 'del',
 13: 'OpenAI.',
 14: 'lista',
 15: 'estimado',
 16: 'fundador,',
 17: 'empresario',
 18: 'abril',
 19: '2022,',
 20: 'índice',
 21: 'real',
 22: 'patrimonio',
 23: 'Company;',
 24: 'el',
 25: 'consejero',
 26: '252',
 27: 'inversor',
 28: 'estadounidense',
 29: 'Es',
 30: 'es',
 31: 'arquitecto',
 32: 'rica',
 33: 'y',
 34: 'delegado',
 35: 'CEO',
 36: 'fundador',
 37: 'la',
 38: 'multimillonarios',
 39: 'e',
 40: 'en',
 41: 'productos',
 42: 'mundo',
 43: 'persona',
 44: 'cofundador',
 45: 'Bloomberg',
 46: 'de',
 47: 'millones',
 48: 'origen',
 49: 'Musk',
 50: 'mil',
 51: 'dólares',
 52: 'más',
 53: 'Boring',
 54: 'Con',
 55: 'tiempo',
 56: 'Forbes.',
 57: 'magnate',
 58: 'Neuralink',
 59: 'The'}

In [3]:
word_to_ix

{'ingeniero': 0,
 'Tesla,': 1,
 'SpaceX;': 2,
 'neto': 3,
 'ángel,': 4,
 'según': 5,
 'sudafricano.': 6,
 'un': 7,
 'jefe': 8,
 'Elon': 9,
 'Inc;': 10,
 'unos': 11,
 'del': 12,
 'OpenAI.': 13,
 'lista': 14,
 'estimado': 15,
 'fundador,': 16,
 'empresario': 17,
 'abril': 18,
 '2022,': 19,
 'índice': 20,
 'real': 21,
 'patrimonio': 22,
 'Company;': 23,
 'el': 24,
 'consejero': 25,
 '252': 26,
 'inversor': 27,
 'estadounidense': 28,
 'Es': 29,
 'es': 30,
 'arquitecto': 31,
 'rica': 32,
 'y': 33,
 'delegado': 34,
 'CEO': 35,
 'fundador': 36,
 'la': 37,
 'multimillonarios': 38,
 'e': 39,
 'en': 40,
 'productos': 41,
 'mundo': 42,
 'persona': 43,
 'cofundador': 44,
 'Bloomberg': 45,
 'de': 46,
 'millones': 47,
 'origen': 48,
 'Musk': 49,
 'mil': 50,
 'dólares': 51,
 'más': 52,
 'Boring': 53,
 'Con': 54,
 'tiempo': 55,
 'Forbes.': 56,
 'magnate': 57,
 'Neuralink': 58,
 'The': 59}

In [5]:

data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

In [7]:

class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()
        
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)


model = CBOW(vocab_size, EMDEDDING_DIM)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# TRAIN
for epoch in range(50):
    total_loss = 0

    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)  

        log_probs = model(context_vector)

        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]]))

    #optimize at the end of each epoch
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

# TEST
context = ['Elon','Musk','un', 'empresario']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Contexto dado: {context}\n')
print(f'Prediccion: {ix_to_word[torch.argmax(a[0]).item()]}')

Raw text: Elon Musk es un empresario inversor y magnate estadounidense de origen sudafricano. Es el fundador, consejero delegado e ingeniero jefe de SpaceX; inversor ángel, CEO y arquitecto de productos de Tesla, Inc; fundador de The Boring Company; y cofundador de Neuralink y OpenAI. Con un patrimonio neto estimado en unos 252 mil millones de dólares en abril de 2022, Musk es la persona más rica del mundo según el índice de multimillonarios de Bloomberg y la lista de multimillonarios en tiempo real de Forbes.

Contexto dado: ['Elon', 'Musk', 'un', 'empresario']

Prediccion: es


In [ ]:
model

In [9]:
model.embeddings.weight[word_to_ix['Elon']].shape

torch.Size([100])

In [11]:
word_to_ix

{'empresario,': 0,
 'origen': 1,
 'consejero': 2,
 'fundador': 3,
 'rica': 4,
 'sudafricano.': 5,
 '252': 6,
 'e': 7,
 'The': 8,
 'dólares': 9,
 '2022,': 10,
 'ingeniero': 11,
 'más': 12,
 'y': 13,
 'Inc;': 14,
 'de': 15,
 'estadounidense': 16,
 'mil': 17,
 'millones': 18,
 'en': 19,
 'multimillonarios': 20,
 'un': 21,
 'persona': 22,
 'tiempo': 23,
 'magnate': 24,
 'Tesla,': 25,
 'Neuralink': 26,
 'Forbes.': 27,
 'ángel,': 28,
 'patrimonio': 29,
 'neto': 30,
 'fundador,': 31,
 'CEO': 32,
 'Con': 33,
 'según': 34,
 'Elon': 35,
 'Es': 36,
 'Company;': 37,
 'lista': 38,
 'cofundador': 39,
 'abril': 40,
 'el': 41,
 'Boring': 42,
 'la': 43,
 'OpenAI.': 44,
 'es': 45,
 'Musk': 46,
 'jefe': 47,
 'mundo': 48,
 'arquitecto': 49,
 'inversor': 50,
 'Bloomberg': 51,
 'índice': 52,
 'delegado': 53,
 'unos': 54,
 'del': 55,
 'real': 56,
 'estimado': 57,
 'SpaceX;': 58,
 'productos': 59}